In [3]:
import gradio as gr
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Load the trained model (ensure the model is in the same directory or provide the correct path)
with open ("final_model_bagg.pkl","rb") as file:
    model = pickle.load(file)

# Function to predict churn
def predict_churn(gender, SeniorCitizen, Partner, Dependents, tenure,
                  PhoneService, MultipleLines, InternetService,
                  OnlineSecurity, OnlineBackup, DeviceProtection,
                  TechSupport, StreamingTV, StreamingMovies, Contract,
                  PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges):
    
    # Convert inputs to the encoded format
    gender = 1 if gender == 'Male' else 0
    Partner = 1 if Partner == 'Yes' else 0
    Dependents = 1 if Dependents == 'Yes' else 0
    PhoneService = 1 if PhoneService == 'Yes' else 0
    MultipleLines = {'No': 0, 'No phone service': 1, 'Yes': 2}[MultipleLines]
    InternetService = {'No': 0, 'DSL': 1, 'Fiber optic': 2}[InternetService]
    OnlineSecurity = {'No': 0, 'No internet service': 1, 'Yes': 2}[OnlineSecurity]
    OnlineBackup = {'No': 0, 'No internet service': 1, 'Yes': 2}[OnlineBackup]
    DeviceProtection = {'No': 0, 'No internet service': 1, 'Yes': 2}[DeviceProtection]
    TechSupport = {'No': 0, 'No internet service': 1, 'Yes': 2}[TechSupport]
    StreamingTV = {'No': 0, 'No internet service': 1, 'Yes': 2}[StreamingTV]
    StreamingMovies = {'No': 0, 'No internet service': 1, 'Yes': 2}[StreamingMovies]
    Contract = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}[Contract]
    PaperlessBilling = 1 if PaperlessBilling == 'Yes' else 0

    # Create input array
    input_data = [[gender, SeniorCitizen, Partner, Dependents, tenure,
                   PhoneService, MultipleLines, InternetService,
                   OnlineSecurity, OnlineBackup, DeviceProtection,
                   TechSupport, StreamingTV, StreamingMovies, Contract,
                   PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges]]

    # Predict churn probability
    pred = model.predict_proba(input_data)[:, 1][0]

    # Return prediction result
    if pred > 0.5:
        return f'This customer is more likely to churn: Probability = {round(pred * 100, 2)}%'
    else:
        return f'This customer is less likely to churn: Probability = {round(pred * 100, 2)}%'

# Define Gradio interface
iface = gr.Interface(
    fn=predict_churn,
    inputs=[
        gr.Radio(['Male', 'Female'], label='Gender'),
        gr.Radio([0, 1], label='Senior Citizen (0=No, 1=Yes)'),
        gr.Radio(['Yes', 'No'], label='Has Partner?'),
        gr.Radio(['Yes', 'No'], label='Has Dependents?'),
        gr.Number(label='Tenure (Months)'),
        gr.Radio(['Yes', 'No'], label='Has Phone Service?'),
        gr.Radio(['No', 'No phone service', 'Yes'], label='Multiple Lines'),
        gr.Radio(['No', 'DSL', 'Fiber optic'], label='Internet Service Type'),
        gr.Radio(['No', 'No internet service', 'Yes'], label='Online Security'),
        gr.Radio(['No', 'No internet service', 'Yes'], label='Online Backup'),
        gr.Radio(['No', 'No internet service', 'Yes'], label='Device Protection'),
        gr.Radio(['No', 'No internet service', 'Yes'], label='Tech Support'),
        gr.Radio(['No', 'No internet service', 'Yes'], label='Streaming TV'),
        gr.Radio(['No', 'No internet service', 'Yes'], label='Streaming Movies'),
        gr.Radio(['Month-to-month', 'One year', 'Two year'], label='Contract Type'),
        gr.Radio(['Yes', 'No'], label='Uses Paperless Billing?'),
        gr.Number(label='Payment Method (Encoded as numeric)'),
        gr.Number(label='Monthly Charges'),
        gr.Number(label='Total Charges'),
    ],
    outputs=gr.Textbox(label='Prediction'),
    title='Telecom Customer Churn Prediction',
    description='This application predicts whether a customer is likely to churn based on their telecom usage and account details.',
    allow_flagging='never'
)

# Run the application
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
